In [0]:
import numpy as np
import pandas as pd
import os
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.utils.random import sample_without_replacement
import matplotlib.pyplot as plt

In [10]:
train_data = pd.read_csv('data/ann-train.data', sep = ' ',
                         header = None)
test_data = pd.read_csv('data/ann-test.data', sep = ' ',
                         header = None)

train_data.drop(train_data.columns[[22,23]], axis = 1,
                inplace = True)
test_data.drop(test_data.columns[[22,23]], axis = 1,
                inplace = True)

X_train = train_data.iloc[:, range(21)]
y_train = train_data.iloc[:, 21]

X_test = test_data.iloc[:, range(21)]
y_test = test_data.iloc[:, 21]

nn = MLPClassifier(hidden_layer_sizes = (5,),
                   activation = 'tanh',
                   solver = 'sgd',
                   max_iter = 5000,
                   random_state = 3)
nn.fit(X_train, y_train)
print(nn.score(X_test, y_test))
print(confusion_matrix(y_test, nn.predict(X_test)))
print(classification_report(y_test, nn.predict(X_test)))

0.927071178529755
[[   0    0   73]
 [   0    0  177]
 [   0    0 3178]]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        73
           2       0.00      0.00      0.00       177
           3       0.93      1.00      0.96      3178

    accuracy                           0.93      3428
   macro avg       0.31      0.33      0.32      3428
weighted avg       0.86      0.93      0.89      3428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
# Finding the amount of node with cross-validation in the hidden layer
parameters = range(2, 11)
cv = KFold(n_splits = 10, shuffle = True, random_state = 4)
parameter_performance = []
for c in parameters:
  cv_performance = []
  for train_index, ver_index in cv.split(X_train, y_train):
    cv_nn = MLPClassifier(hidden_layer_sizes = (c, ),
                          activation = 'tanh',
                          solver = 'sgd',
                          max_iter = 500,
                          learning_rate_init = 0.01,
                          random_state = 5)
    
    cv_nn.fit(X_train.iloc[train_index, :],
              y_train.iloc[train_index])
    accuracy = cv_nn.score(X_train.iloc[ver_index, :],
                           y_train.iloc[ver_index])
    cv_performance.append(accuracy)

  cv_performance = np.array(cv_performance)
  parameter_performance.append(cv_performance.mean())

best_neuron_count = parameters[np.argmax(parameter_performance)]

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [11]:
best_nn = MLPClassifier(hidden_layer_sizes = best_neuron_count,
                        activation = 'tanh',
                        solver = 'sgd',
                        max_iter = 500,
                        learning_rate_init = 0.01,
                        random_state = 5)

best_nn.fit(X_test, y_test)
accuracy_test = best_nn.score(X_test, y_test)
print(accuracy_test)
print(confusion_matrix(y_test, best_nn.predict(X_test)))
print(classification_report(y_test, best_nn.predict(X_test)))

0.9384480746791132
[[  47    5   21]
 [   0    0  177]
 [   7    1 3170]]
              precision    recall  f1-score   support

           1       0.87      0.64      0.74        73
           2       0.00      0.00      0.00       177
           3       0.94      1.00      0.97      3178

    accuracy                           0.94      3428
   macro avg       0.60      0.55      0.57      3428
weighted avg       0.89      0.94      0.91      3428



/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [14]:
index_1 = np.where(y_train == 1)
index_2 = np.where(y_train == 2)
index_3 = np.where(y_train == 3)

example2 = sample_without_replacement(n_population = sum(y_train == 2),
                                      n_samples = sum(y_train == 1))
example3 = sample_without_replacement(n_population = sum(y_train == 3),
                                      n_samples = sum(y_train == 1))

y_train_balanced = y_train[index_1[0]]
y_train_balanced = np.append(y_train_balanced,
                             y_train[index_2[0][example2]])
y_train_balanced = np.append(y_train_balanced,
                             y_train[index_3[0][example3]])
balanced_indexes = np.append(np.array(index_1),
                             np.array(index_2[0][example2]))
balanced_indexes = np.append(balanced_indexes,
                             np.array(index_3[0][example3]))
X_train_balanced = X_train.iloc[balanced_indexes, :]

balanced_nn = MLPClassifier(hidden_layer_sizes = best_neuron_count,
                                activation = 'tanh',
                                solver = 'sgd',
                                learning_rate_init = 0.01,
                                max_iter = 500,
                                random_state = 5)

balanced_nn.fit(X_train_balanced, y_train_balanced)
balanced_accuracy = balanced_nn.score(X_test, y_test)
print(balanced_accuracy)
print(confusion_matrix(y_test, balanced_nn.predict(X_test)))
print(classification_report(y_test, balanced_nn.predict(X_test)))

0.6796966161026837
[[  63    9    1]
 [   3  115   59]
 [  51  975 2152]]
              precision    recall  f1-score   support

           1       0.54      0.86      0.66        73
           2       0.10      0.65      0.18       177
           3       0.97      0.68      0.80      3178

    accuracy                           0.68      3428
   macro avg       0.54      0.73      0.55      3428
weighted avg       0.92      0.68      0.76      3428



/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
